<!--NOTEBOOK_HEADER-->
*This notebook contains material from [cbe30338-2021](https://jckantor.github.io/cbe30338-2021);
content is available [on Github](https://github.com/jckantor/cbe30338-2021.git).*


<!--NAVIGATION-->
< [5.3 Linear Production Model in Pyomo](https://jckantor.github.io/cbe30338-2021/05.03-Linear-Production-Model-in-Pyomo.html) | [Contents](toc.html) | [Tag Index](tag_index.html) | [5.6 Design of a Cold Weather Fuel](https://jckantor.github.io/cbe30338-2021/05.06-Design-of-a-Cold-Weather-Fuel.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe30338-2021/blob/master/docs/05.05-Linear-Blending-Problem.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe30338-2021/05.05-Linear-Blending-Problem.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

# 5.5 Linear Blending Problem

## 5.5.1 Problem Statement (Jenchura, 2017)

A brewery receives an order for 100 gallons of 4% ABV (alchohol by volume) beer. The brewery has on hand beer A that is 4.5% ABV that cost \\$0.32 per gallon to make, and beer B that is 3.7% ABV and cost \\$0.25 per gallon. Water can also be used as a blending agent at a cost of \\$0.05 per gallon. Find the minimum cost blend that meets the customer requirements.

## 5.5.2 Solving optimization problems with CVXPY

The blending problem described above is relatively simple, it can be solved in CVXPY using no more than `cp.Variable()`, `cp.Minimize()`, and `cp.Problem()` as demonstrated below.

### 5.5.2.1 Step 1. Coding Problem Data as a Python Dictionary

The first step is to represent the problem data in a generic manner that could be extended to include additional blending components.  Here we use a dictionary of raw materials, each key denoting a unique blending agent. For each key there is a second level dictionary containing attributes of the blending component. This nested "dictionary of dictionaries" organization is a useful of organizing tabular data for optimization problems.

In [99]:
data = {
    'A': {'abv': 0.045, 'cost': 0.32},
    'B': {'abv': 0.037, 'cost': 0.25},
    'W': {'abv': 0.000, 'cost': 0.05},
}
print(data)

{'A': {'abv': 0.045, 'cost': 0.32}, 'B': {'abv': 0.037, 'cost': 0.25}, 'W': {'abv': 0.0, 'cost': 0.05}}


### 5.5.2.2 Step 2. Identifying index sets

The objectives and constraints encountered in optimization problems often include sums over a set of objects. In the case, we will need to create sums over the set of raw materials in the blending problem.

In [100]:
components = set(data.keys())
print(components)

{'B', 'W', 'A'}


### 5.5.2.3 Step 3. Create decision variables

In [101]:
x = {c: cp.Variable() for c in components}
print(x)

{'B': Variable(()), 'W': Variable(()), 'A': Variable(())}


### 5.5.2.4 Step 4. Objective Function

If we let subscript $c$ denote a blending component from the set of blending components $C$, and denote the volume of $c$ used in the blend as $x_c$, the cost of the blend is

\begin{align}
\mbox{cost} & = \sum_{c\in C} x_c P_c
\end{align}

where $P_c$ is the price per unit volume of $c$. Using the Python data dictionary defined above, the price $P_c$ is given by `data[c]['cost']`.

In [102]:
total_cost = sum([x[c]*data[c]["cost"] for c in components])
objective = cp.Minimize(total_cost)
print(objective)

minimize var2566 @ 0.25 + var2567 @ 0.05 + var2568 @ 0.32


### 5.5.2.5 Step 5. Constraints

#### 5.5.2.5.1 Volume Constraint

The customer requirement is produce a total volume $V$. Assuming ideal solutions, the constraint is given by

\begin{align}
V &  = \sum_{c\in C} x_c
\end{align}

where $x_c$ denotes the volume of component $c$ used in the blend.

#### 5.5.2.5.2 Product Composition Constraint

The product composition is specified as 4% alchohol by volume. Denoting this as $\bar{A}$, the constraint may be written as

\begin{align}
\bar{A} & = \frac{\sum_{c\in C}x_c A_c}{\sum_{c\in C} x_c}
\end{align}

where $A_c$ is the alcohol by volume for component $c$. As written, this is a nonlinear constraint. Multiplying both sides of the equation by the denominator yields a linear constraint

\begin{align}
\bar{A}\sum_{c\in C} x_c & = \sum_{c\in C}x_c A_c
\end{align}

A final form for this constraint can be given in either of two versions. In the first version we subtract the left-hand side from the right to give

\begin{align}
0 & = \sum_{c\in C}x_c \left(A_c - \bar{A}\right) & \mbox{ Version 1 of the linear blending constraint}
\end{align}

Alternatively, the summation on the left-hand side corresponds to total volume. Since that is known as part of the problem specification, the blending constraint could also be written as

\begin{align}
\bar{A}V & = \sum_{c\in C}x_c A_c  & \mbox{ Version 2 of the linear blending constraint}
\end{align}

Which should you use? Either will generally work well. The advantage of version 1 is that it is fully specified by a product requirement $\bar{A}$, which is sometimes helpful in writing elegant Python code.

### 5.5.2.6 CVXPY Version 1

For this first version, we will write a CVXPY model that uses a minimal of Python or CVXPY coding features.

In [67]:
import numpy as np
import cvxpy as cp

vol = 100
abv = 0.04

A = cp.Variable(nonneg=True)
B = cp.Variable(nonneg=True)
W = cp.Variable(nonneg=True)

constraints = [
    A + B + W == vol,
    A*(blending_data["A"]["abv"] - abv) + B*(blending_data["B"]["abv"] - abv) + W*(blending_data["W"]["abv"]) == 0
]

cost = A*blending_data["A"]["cost"] + B*blending_data["A"]["cost"] + W*blending_data["A"]["cost"]

problem = cp.Problem(cp.Minimize(cost), constraints)
problem.solve()

print(f"cost to produce {vol} gallons at abv {abv}: ${cost.value:4.2f}")
print(f"A = {A.value:4.2f} gallons")
print(f"B = {B.value:4.2f} gallons")
print(f"W = {W.value:4.2f} gallons")

cost to produce 100 gallons at abv 0.04: $32.00
A = 24.49 gallons
B = 40.82 gallons
W = 34.69 gallons


### 5.5.2.7 CVXPY Version 2: Using dictionaries

Version 1 of our model solves the problem at hand but would require rewriting should any aspect of the problem statement be changed. Consider, for example, what modifications would be needed if

* One or more additional blendinig stocks become available. Perhaps many more.
* We wish to specify additional properties of the blended product.

In [68]:
import numpy as np
import cvxpy as cp

vol = 100
abv = 0.04

components = blending_data.keys()
x = dict()
for c in components:
    x[c] = cp.Variable(nonneg=True)

x

{'A': Variable((), nonneg=True),
 'B': Variable((), nonneg=True),
 'W': Variable((), nonneg=True)}

In [37]:
import numpy as np
import cvxpy as cp

def optimal_blend(product_volume, product_abv, component_blending_data):
    
    components = component_blending_data.keys()
    
    x = {c: cp.Variable(nonneg=True, name=c) for c in components}
    
    constraints = [
        sum(x[c] for c in components) == product_volume,
        sum(x[c]*(component_blending_data[c]['abv'] - product_abv) for c in components) == 0
    ]
    
    objective = cp.Minimize(sum(x[c]*component_blending_data[c]['cost'] for c in components))
    
    problem = cp.Problem(objective, constraints)
    problem.solve()
    return problem
    
soln = optimal_blend(100, 0.04, component_blending_data)


for var in soln.variables():
    print(var, var.value)

A 37.500001136050194
B 62.49999861831747
W 2.4563236418836055e-07


#### 5.5.2.7.1 Pyomo Model

A Pyomo implementation of this blending model is shown in the next cell. The model is contained within a Python function so that it can be more easily reused for additional calculations, or eventually for use by the process operator.

Note that the pyomo library has been imported with the prefix `pyomo`. This is good programming practive to avoid namespace collisions with problem data.

In [19]:
import pyomo.environ as pyomo

vol = 100
abv = 0.040

def beer_blend(vol, abv, data):
    
    C = data.keys()
    
    model = pyomo.ConcreteModel()
    
    model.x = pyomo.Var(C, domain=pyomo.NonNegativeReals)
    
    model.cost = pyomo.Objective(expr = sum(model.x[c]*data[c]['cost'] for c in C))
    
    model.vol = pyomo.Constraint(expr = vol == sum(model.x[c] for c in C))
    model.abv = pyomo.Constraint(expr = 0 == sum(model.x[c]*(data[c]['abv'] - abv) for c in C))

    solver = pyomo.SolverFactory('glpk')
    solver.solve(model)

    print('Optimal Blend')
    for c in data.keys():
        print('  ', c, ':', model.x[c](), 'gallons')
    print()
    print('Volume = ', model.vol(), 'gallons')
    print('Cost = $', model.cost())
    
beer_blend(vol, abv, data)

Optimal Blend
   A : 37.5 gallons
   B : 62.5 gallons
   W : 0.0 gallons

Volume =  100.0 gallons
Cost = $ 27.625


<!--NAVIGATION-->
< [5.3 Linear Production Model in Pyomo](https://jckantor.github.io/cbe30338-2021/05.03-Linear-Production-Model-in-Pyomo.html) | [Contents](toc.html) | [Tag Index](tag_index.html) | [5.6 Design of a Cold Weather Fuel](https://jckantor.github.io/cbe30338-2021/05.06-Design-of-a-Cold-Weather-Fuel.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe30338-2021/blob/master/docs/05.05-Linear-Blending-Problem.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe30338-2021/05.05-Linear-Blending-Problem.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>